In [1]:
import csv
import sys
import io
import re # 정규표현식 패키지
from kiwipiepy import Kiwi, Option
import tomotopy as tp
import pandas as pd

origin_path = '201910.csv'
result_path = "201910_topic.csv"

origin=pd.read_csv(origin_path,encoding='utf-8')

text=origin['text']
print(text)

with open('News_Contents.txt','w',encoding='utf-8') as file:
    for item in text:
        file.write("%s\n" % item)

class Reader: #토픽 학습에 사용될 callable한 class
    def __init__(self, filePath):
        self.file = open(filePath,'r',encoding='UTF-8')
    
    def read(self, id):
        if id == 0: self.file.seek(0)
        return self.file.readline()

reader = Reader('News_Contents.txt')



0        신협중앙회는 지난 27일부터 29일까지 말레이시아 쿠알라룸프루 소재 버자야 타임스스...
1        스스로 돈을 찍어 예산을 충당하는 한국은행이 수년간 1%대 초저금리로 직원들에게 주...
2        주요 공공기관의 친인척 채용비리 의혹이 대부분 사실인 것으로 드러났다. 서울교통공사...
3        재판에서 실형을 선고받고 잠적한 불구속 피고인을 올해 절반도 검거하지 못한 것으로 ...
4        관세청이 기관 포상금으로 책정된 예산을 내부 쌈짓돈으로 전용해온 것으로 드러났다. ...
                               ...                        
29633    매년 2월 초 뛰어난 수학 능력을 지닌 미국 전역의 고교생들은 물론, 캐나다, 중국...
29634    영어 변별력이 약해진 요즘 입시에서 수학과 국어는 입시 당락을 좌우한다고 할 만큼 ...
29635    지난 10월 24일(목) 오후 7시, 세화여자고등학교(교장 김흥기) 강당에서 ‘20...
29636    지난 10월 28일(월) 오후 7시, 단국대학교사범대학부속고등학교(교장 장준성) 강...
29637    지난 23일(수) 오후 6시 경기고등학교 문화관에서 입학설명회가 열렸다. 최근 고교...
Name: text, Length: 29638, dtype: object
['것', '수', '이', '나', '사람', '등', '우리', '때', '년', '말', '일', '때문', '그것', '일', '문제', '사회', '중', '씨', '지금', '속', '하나', '집', '월', '데', '자신', '내', '경우', '명', '생각', '시간', '그녀', '앞', '번', '여자', '개', '전', '사실', '점', '정도', '원', '소리']
End
             date press category                                  headline  \
0    

In [ ]:
#rm_stopword

stopword=[]

def rm_stopword(stopword_file): 
    files = open(stopword_file, 'r', encoding="utf-8")
    while True: #불용어 리스트 생성
        line = files.readline()
        if not line: break
        wordlist = line.split('\t')
        if (wordlist[1].startswith('N')):
            stopword.append(wordlist[0])
    print(stopword)      
    stopwords = set(stopword)
    close(files)
    return stopword

In [ ]:
#Morp_analysis

filename="News_Contents.txt"  #파일 경로
file = open(filename, 'r', encoding="UTF-8")
kiwi=Kiwi()
kiwi.extract_add_words(reader.read) #단어 학습
kiwi.prepare()



def Morp_Analysis(sent): # 파일의 라인을 분석할 tokenize 함수
    res, score = kiwi.analyze(sent)[0] # 첫번째 결과를 사용한다, 분석할때 나오는 결과에서 단어만 추출
    return [word
            for word, tag, _, _ in res
            if tag.startswith('N') and word not in rm_stopword("한국어불용어100.txt")] #불용어사전 적용
            

In [ ]:
#exract_topic

#LDA 모델을 적용해서 토픽 추출, k는 topic 개수
#alpha는 문헌-토픽 디리클레 분포의 하이퍼 파라미터
#eta는 토픽-단어 디리클레 분포의 하이퍼 파라미터 두개다 상수인듯하다
#min_cf는 단어의 최소 장서 빈도수, 전체 문헌내 출현빈도
#min_df는 단어의 최서 문헌 빈도수, 출현한 문헌 숫자수 의미
#tw는 용어 가중치 기법으로, ONE, IDF, PMI를 사용가능, ONE 보다는 PMI나 IDF 둘중 하나 사용 

file = open(result_path, "w", encoding='utf-8', newline = '')
writer = csv.writer(file)
writer.writerow(['index', 'topics'])


def extract_topic(extracted_topic):
    model = tp.LDAModel(k=1, alpha=0.1, eta=0.001, min_cf=3,min_df=1, tw=tp.TermWeight.PMI)
    model.add_doc(extracted_topic) #추출하고 모델안의 문헌을 넣는다. 즉, 학습과정에 쓰일 문헌을 생성
    model.train(0) #학습 초기화
    s = ""
    for j in range(0,100):
        model.train(20) #문헌 학습, 안에 숫자는 깁스 샘플링의 반복횟수
		#이때 기본값으로 시스템내 가용한 모든 스레드의 개수사용, 그리고 병렬화 방법을 찾아서 실행시켜준다
    for k in range(model.k): #k는 토픽의 개수
        res = model.get_topic_words(k, top_n=10) #하위 토픽 i에 해당하는 top_n개의 단어 반환
        s = ', '.join(w for w, p in res)
    writer.writerow([i, s])
    # print(i, s)


for i, line in enumerate(open(filename, encoding='utf-8')): #해당 경로의 파일을 받아와 한 라인씩 model에 추가
    extract_topic(Morp_Analysis(line))
file.close()
print("End")

topic = pd.read_csv(result_path, encoding='utf-8')

#토픽 추출 후 기존 csv에 병합해서 출력

merge = pd.concat([origin, topic['topics']], axis=1)
print(merge)
merge.to_csv('201910.csv', index=False, header=True,encoding="utf-8-sig")
print('merge topics')
